In [47]:
import glob
import pickle
import numpy
import h5py
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [48]:
def train_network():
    """ Train a Neural Network to generate music """
    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

In [64]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("midi_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    #stream1 = stream.Stream(notes)
    #stream1.show('midi')
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)
    stream1 = stream.Stream()
    for i in notes:
        try:
            stream1.append(note.Note(i))
        except:
            stream1
    stream1.show('midi')
        
    return notes



In [65]:
get_notes()

Parsing midi_songs/Ff7-Jenova_Absolute.mid
Parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/DOS.mid
Parsing midi_songs/FF3_Battle_(Piano).mid
Parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/ff7themep.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/Finalfantasy5gilgameshp.mid
Parsing midi_songs/Cids.mid
Parsing midi_songs/lurk_in_dark.mid
Parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing midi_songs/ff4pclov.mid
Parsing midi_songs/AT.mid
Parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing midi_songs/8.mid
Parsing midi_songs/sobf.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/waltz_de_choco.mid
Pars

['F3',
 'F3',
 'B-4',
 'F3',
 'G4',
 'G#4',
 'F4',
 'F3',
 'G4',
 'G3',
 'E-4',
 'G#3',
 'F3',
 'F5',
 'G5',
 'G#5',
 'F3',
 'B-5',
 'G5',
 'G#5',
 'F3',
 'B-5',
 'C6',
 'B-5',
 'F3',
 'E-6',
 'C#4',
 'F6',
 'C4',
 'F3',
 'F3',
 'B-4',
 'F3',
 'G4',
 'G#4',
 'F4',
 'F3',
 'G4',
 'G3',
 'E-4',
 'G#3',
 'F3',
 'F5',
 'G5',
 'G#5',
 'F3',
 'B-5',
 'G5',
 'G#5',
 'F3',
 'B-5',
 'C6',
 'B-5',
 'F3',
 'E-6',
 'E-3',
 'F6',
 'E3',
 'F2',
 'F2',
 'B-4',
 'F2',
 'G4',
 'G#4',
 'B-2',
 'F4',
 'G4',
 'C3',
 'E-4',
 'F2',
 'F5',
 'G5',
 'G#5',
 'F2',
 'B-5',
 'G5',
 'G#5',
 'F2',
 'B-5',
 'C6',
 'B-5',
 'F2',
 'E-6',
 'C#3',
 'F6',
 'C3',
 'F2',
 'F2',
 'B-4',
 'F2',
 'G4',
 'G#4',
 'B-2',
 'F4',
 'G4',
 'C3',
 'E-4',
 'F2',
 'F5',
 'G5',
 'G#5',
 'F2',
 'B-5',
 'G5',
 'G#5',
 'F2',
 'B-5',
 'C6',
 'B-5',
 'F2',
 'E-6',
 'E-2',
 'F6',
 'E2',
 '0.5',
 'F2',
 'F2',
 'F2',
 'B-2',
 'C3',
 '8.0',
 'F2',
 'F2',
 '7.10',
 'F2',
 'F2',
 'C#3',
 'C3',
 '0.3',
 'F2',
 'F2',
 'F2',
 'B-2',
 'C3',
 '10.2',
 

In [51]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    ) 
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=1, batch_size=64, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()

Parsing midi_songs/Ff7-Jenova_Absolute.mid
Parsing midi_songs/FF8_Shuffle_or_boogie_pc.mid
Parsing midi_songs/ahead_on_our_way_piano.mid
Parsing midi_songs/balamb.mid
Parsing midi_songs/DOS.mid
Parsing midi_songs/FF3_Battle_(Piano).mid
Parsing midi_songs/Suteki_Da_Ne_(Piano_Version).mid
Parsing midi_songs/caitsith.mid
Parsing midi_songs/sera_.mid
Parsing midi_songs/Oppressed.mid
Parsing midi_songs/ff4-airship.mid
Parsing midi_songs/ff7themep.mid
Parsing midi_songs/FFVII_BATTLE.mid
Parsing midi_songs/electric_de_chocobo.mid
Parsing midi_songs/Ff7-One_Winged.mid
Parsing midi_songs/Finalfantasy5gilgameshp.mid
Parsing midi_songs/Cids.mid
Parsing midi_songs/lurk_in_dark.mid
Parsing midi_songs/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
Parsing midi_songs/ff4pclov.mid
Parsing midi_songs/AT.mid
Parsing midi_songs/Final_Fantasy_7_-_Judgement_Day_Piano.mid
Parsing midi_songs/8.mid
Parsing midi_songs/sobf.mid
Parsing midi_songs/In_Zanarkand.mid
Parsing midi_songs/waltz_de_choco.mid
Pars

In [62]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(64))
    model.add(Dense(64))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights-improvement-01-4.7668-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    #for i  in network_input:
    #    print(i)
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)
    
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))


    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        start = numpy.random.randint(0, len(network_input)-1)
        pattern = network_input[start]
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        #print(prediction_input)
        
        ##Our model is stupid.
        prediction = model.predict(prediction_input, verbose=1)
        #print(prediction)

        index = numpy.argmax(prediction)
        print(index)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            #print(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            #print(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output2.mid')
    

if __name__ == '__main__':
    generate()

1/1 [==============================] - 4s 4s/step
331
1/1 [==============================] - 0s 72ms/step
331
1/1 [==============================] - 0s 71ms/step
331
1/1 [==============================] - 0s 76ms/step
331
1/1 [==============================] - 0s 132ms/step
331
1/1 [==============================] - 0s 116ms/step
331
1/1 [==============================] - 0s 104ms/step
331
1/1 [==============================] - 0s 72ms/step
331
1/1 [==============================] - 0s 79ms/step
331
1/1 [==============================] - 0s 122ms/step
331
1/1 [==============================] - 0s 110ms/step
331
1/1 [==============================] - 0s 78ms/step
331
1/1 [==============================] - 0s 71ms/step
331
1/1 [==============================] - 0s 62ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 68ms/step
331
1/1 [==============================] - 0s 117

331
1/1 [==============================] - 0s 72ms/step
331
1/1 [==============================] - 0s 63ms/step
331
1/1 [==============================] - 0s 57ms/step
331
1/1 [==============================] - 0s 57ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 62ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 63ms/step
331
1/1 [==============================] - 0s 70ms/step
331
1/1 [==============================] - 0s 72ms/step
331
1/1 [==============================] - 0s 70ms/step
331
1/1 [==============================] - 0s 69ms/step
331
1/1 [==============================] - 0s 69ms/step
331
1/1 [==============================] - 0s 69ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 74

331
1/1 [==============================] - 0s 57ms/step
331
1/1 [==============================] - 0s 52ms/step
331
1/1 [==============================] - 0s 51ms/step
331
1/1 [==============================] - 0s 57ms/step
331
1/1 [==============================] - 0s 62ms/step
331
1/1 [==============================] - 0s 62ms/step
331
1/1 [==============================] - 0s 55ms/step
331
1/1 [==============================] - 0s 53ms/step
331
1/1 [==============================] - 0s 56ms/step
331
1/1 [==============================] - 0s 61ms/step
331
1/1 [==============================] - 0s 63ms/step
331
1/1 [==============================] - 0s 57ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 64ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 63

331
1/1 [==============================] - 0s 64ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 65ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 61ms/step
331
1/1 [==============================] - 0s 68ms/step
331
1/1 [==============================] - 0s 67ms/step
331
1/1 [==============================] - 0s 63ms/step
331
1/1 [==============================] - 0s 64ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 61ms/step
331
1/1 [==============================] - 0s 63ms/step
331
1/1 [==============================] - 0s 67ms/step
331
1/1 [==============================] - 0s 59ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 61ms/step
331
1/1 [==============================] - 0s 60ms/step
331
1/1 [==============================] - 0s 62

In [26]:
filename = 'weights.hdf5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

Keys: KeysView(<HDF5 file "weights.hdf5" (mode r)>)


In [37]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('weights-improvement-04-4.7016-bigger.hdf5')

    return model

def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        print(index)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')

if __name__ == '__main__':
    generate()

283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
283
